In [1]:
import numpy as np
import xgboost as xgb
from wideboost.wrappers import wxgb
from wideboost.wrappers.wxgb import wxgbModel

from matplotlib import pyplot as plt

In [2]:
import pandas as pd

train = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.data'))
test = np.asarray(pd.read_csv('/Users/michaelhorrell/Downloads/adult.test'))

In [3]:
ntrain = train.shape[0]
full = np.concatenate([train,test],axis=0)

c0 = full[:,0].astype(float).reshape([-1,1])
c1 = np.asarray(pd.get_dummies(full[:,1]))
c2 = full[:,2].astype(float).reshape([-1,1])
c3 = np.asarray(pd.get_dummies(full[:,3]))
c4 = full[:,4].astype(float).reshape([-1,1])
c5 = np.asarray(pd.get_dummies(full[:,5]))
c6 = np.asarray(pd.get_dummies(full[:,6]))
c7 = np.asarray(pd.get_dummies(full[:,7]))
c8 = np.asarray(pd.get_dummies(full[:,8]))
c9 = np.asarray(pd.get_dummies(full[:,9]))
c10 = full[:,10].astype(float).reshape([-1,1])
c11 = full[:,11].astype(float).reshape([-1,1])
c12 = full[:,12].astype(float).reshape([-1,1])
c13 = np.asarray(pd.get_dummies(full[:,13]))

label = np.zeros([full.shape[0],1])
label[full[:,14] == ' >50K',0] = 1
label[full[:,14] == ' >50K.',0] = 1

XFULL = np.concatenate([c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13],axis=1)
YFULL = label

xtrain = XFULL[0:ntrain,:]
ytrain = label[0:ntrain,0:1]

xtest = XFULL[ntrain:,:]
ytest = label[ntrain:,:]

dtrain = xgb.DMatrix(xtrain,label=ytrain)
dtest = xgb.DMatrix(xtest,label=ytest)

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    wbst = wxgb.train(param, dtrain, 100,watchlist,evals_result=ed1_results)
    
    if min(ed1_results['test']['error']) < best_val:
        print("NEW BEST VALUE!")
        best_val = min(ed1_results['test']['error'])
    
    return {'loss': min(ed1_results['test']['error']), 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['I']),
    'extra_dims': hp.choice('extra_dims',[0]),
    'objective': hp.choice('objective',['binary:logistic']),
    'eval_metric':hp.choice('eval_metric',[['error']]),
    'eta': hp.loguniform('eta', -7, 0),
    'max_depth' : hp.choice('max_depth',range(1,11)),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1),
    'min_child_weight': hp.loguniform('min_child_weight', -16, 5),
    'alpha': hp.choice('alpha', [0, hp.loguniform('alpha_positive', -16, 2)]),
    'lambda': hp.choice('lambda', [0, hp.loguniform('lambda_positive', -16, 2)]),
    'gamma': hp.choice('gamma', [0, hp.loguniform('gamma_positive', -16, 2)])
}

best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)



{'alpha': 0.5009343724491095, 'btype': 'I', 'colsample_bylevel': 0.7719232236755218, 'colsample_bytree': 0.5537277526779593, 'eta': 0.8419982024663906, 'eval_metric': ('error',), 'extra_dims': 0, 'gamma': 1.4343083798148126, 'lambda': 0.30234979111501825, 'max_depth': 4, 'min_child_weight': 2.731453249460237e-07, 'objective': 'binary:logistic', 'subsample': 0.8953109171365075}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `obj` in train.
Taking first argument of eval_metric. Multiple evals not supported using xgboost backend.
Moving param `eval_metric` to an feval.                
Setting param `disable_default_eval_metric` to 1.      
[10:18:02] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { btype } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificat

[4]	train-error:0.16563	test-error:0.16450                             

[5]	train-error:0.14960	test-error:0.15049                             

[6]	train-error:0.15292	test-error:0.15203                             

[7]	train-error:0.15528	test-error:0.15454                             

[8]	train-error:0.15528	test-error:0.15454                             

[9]	train-error:0.15571	test-error:0.15448                             

[10]	train-error:0.15584	test-error:0.15479                            

[11]	train-error:0.15565	test-error:0.15442                            

[12]	train-error:0.15534	test-error:0.15448                            

[13]	train-error:0.15316	test-error:0.15270                            

[14]	train-error:0.15267	test-error:0.15246                            

[15]	train-error:0.15276	test-error:0.15233                            

[16]	train-error:0.15513	test-error:0.15418                            

[17]	train-error:0.15442	test-error:0.15369        

[1]	train-error:0.14536	test-error:0.14674                             

[2]	train-error:0.14198	test-error:0.14294                             

[3]	train-error:0.14183	test-error:0.14300                             

[4]	train-error:0.14251	test-error:0.14441                             

[5]	train-error:0.14122	test-error:0.14208                             

[6]	train-error:0.14168	test-error:0.14275                             

[7]	train-error:0.14202	test-error:0.14238                             

[8]	train-error:0.14198	test-error:0.14238                             

[9]	train-error:0.14205	test-error:0.14251                             

[10]	train-error:0.14217	test-error:0.14263                            

[11]	train-error:0.14174	test-error:0.14257                            

[12]	train-error:0.14189	test-error:0.14214                            

[13]	train-error:0.14082	test-error:0.14091                            

[14]	train-error:0.14158	test-error:0.14226        

[0]	train-error:0.14607	test-error:0.14902                             

[1]	train-error:0.13750	test-error:0.14404                             

[2]	train-error:0.13443	test-error:0.14072                             

[3]	train-error:0.13160	test-error:0.13618                             

[4]	train-error:0.13139	test-error:0.13655                             

[5]	train-error:0.13090	test-error:0.13606                             

[6]	train-error:0.13025	test-error:0.13452                             

[7]	train-error:0.13077	test-error:0.13440                             

[8]	train-error:0.13117	test-error:0.13378                             

[9]	train-error:0.13194	test-error:0.13495                             

[10]	train-error:0.13194	test-error:0.13544                            

[11]	train-error:0.13185	test-error:0.13470                            

[12]	train-error:0.13179	test-error:0.13483                            

[13]	train-error:0.13102	test-error:0.13495        

[0]	train-error:0.18630	test-error:0.18520                             

[1]	train-error:0.17119	test-error:0.16966                             

[2]	train-error:0.15577	test-error:0.15590                             

[3]	train-error:0.16837	test-error:0.16689                             

[4]	train-error:0.16007	test-error:0.16063                             

[5]	train-error:0.15983	test-error:0.16032                             

[6]	train-error:0.15565	test-error:0.15577                             

[7]	train-error:0.16020	test-error:0.16087                             

[8]	train-error:0.15590	test-error:0.15602                             

[9]	train-error:0.15577	test-error:0.15571                             

[10]	train-error:0.15577	test-error:0.15577                            

[11]	train-error:0.15577	test-error:0.15584                            

[12]	train-error:0.15577	test-error:0.15577                            

[13]	train-error:0.15577	test-error:0.15571        

...

[65]	train-error:0.09770	test-error:0.12887                             

[66]	train-error:0.09745	test-error:0.12887                             

[67]	train-error:0.09736	test-error:0.12875                             

[68]	train-error:0.09724	test-error:0.12887                             

[69]	train-error:0.09733	test-error:0.12887                             

[70]	train-error:0.09705	test-error:0.12893                             

[71]	train-error:0.09530	test-error:0.12905                             

[72]	train-error:0.09450	test-error:0.12912                             

[73]	train-error:0.09407	test-error:0.12930                             

[74]	train-error:0.09386	test-error:0.12936                             

[75]	train-error:0.09315	test-error:0.12948                             

[76]	train-error:0.09269	test-error:0.12936                             

[77]	train-error:0.09278	test-error:0.12948                             

[78]	train-error:0.09275	test-error:0.

[60]	train-error:0.08421	test-error:0.13366                             

[61]	train-error:0.08329	test-error:0.13354                             

[62]	train-error:0.08163	test-error:0.13495                             

[63]	train-error:0.08157	test-error:0.13581                             

[64]	train-error:0.08130	test-error:0.13532                             

[65]	train-error:0.08102	test-error:0.13563                             

[66]	train-error:0.08059	test-error:0.13612                             

[67]	train-error:0.08059	test-error:0.13612                             

[68]	train-error:0.08034	test-error:0.13642                             

[69]	train-error:0.08031	test-error:0.13642                             

[70]	train-error:0.07930	test-error:0.13587                             

[71]	train-error:0.07850	test-error:0.13642                             

[72]	train-error:0.07792	test-error:0.13772                             

[73]	train-error:0.07761	test-error:0.

[56]	train-error:0.12316	test-error:0.12721                             

[57]	train-error:0.12285	test-error:0.12715                             

[58]	train-error:0.12288	test-error:0.12697                             

[59]	train-error:0.12257	test-error:0.12690                             

[60]	train-error:0.12224	test-error:0.12617                             

[61]	train-error:0.12181	test-error:0.12604                             

[62]	train-error:0.12131	test-error:0.12611                             

[63]	train-error:0.12104	test-error:0.12592                             

[64]	train-error:0.12101	test-error:0.12611                             

[65]	train-error:0.12076	test-error:0.12574                             

[66]	train-error:0.12039	test-error:0.12543                             

[67]	train-error:0.12048	test-error:0.12525                             

[68]	train-error:0.12030	test-error:0.12543                             

[69]	train-error:0.12009	test-error:0.

[51]	train-error:0.13102	test-error:0.13335                             

[52]	train-error:0.13096	test-error:0.13292                             

[53]	train-error:0.13071	test-error:0.13292                             

[54]	train-error:0.13068	test-error:0.13286                             

[55]	train-error:0.13065	test-error:0.13274                             

[56]	train-error:0.13047	test-error:0.13274                             

[57]	train-error:0.13056	test-error:0.13274                             

[58]	train-error:0.13034	test-error:0.13274                             

[59]	train-error:0.13028	test-error:0.13262                             

[60]	train-error:0.13019	test-error:0.13237                             

[61]	train-error:0.13004	test-error:0.13243                             

[62]	train-error:0.13022	test-error:0.13212                             

[63]	train-error:0.13022	test-error:0.13206                             

[64]	train-error:0.13013	test-error:0.

[46]	train-error:0.09244	test-error:0.13176                             

[47]	train-error:0.09223	test-error:0.13145                             

[48]	train-error:0.09220	test-error:0.13157                             

[49]	train-error:0.09155	test-error:0.13194                             

[50]	train-error:0.09051	test-error:0.13157                             

[51]	train-error:0.08990	test-error:0.13212                             

[52]	train-error:0.08851	test-error:0.13268                             

[53]	train-error:0.08830	test-error:0.13256                             

[54]	train-error:0.08729	test-error:0.13311                             

[55]	train-error:0.08704	test-error:0.13311                             

[56]	train-error:0.08686	test-error:0.13372                             

[57]	train-error:0.08667	test-error:0.13360                             

[58]	train-error:0.08618	test-error:0.13317                             

[59]	train-error:0.08618	test-error:0.

In [5]:
print(best_val)
print(space_eval(spc, best))

0.123526
{'alpha': 0.008872399312554638, 'btype': 'I', 'colsample_bylevel': 0.9391618513183227, 'colsample_bytree': 0.5695357314506463, 'eta': 0.10265477873863532, 'eval_metric': ('error',), 'extra_dims': 0, 'gamma': 1.2801633181758363e-07, 'lambda': 0.006334906923642443, 'max_depth': 6, 'min_child_weight': 0.07718081627272329, 'objective': 'binary:logistic', 'subsample': 0.803506850739998}
